## SANS2D reduction
This is a notebook to reduce data from SANS2D. 

### Basic components 
* File loading: sample, sample trans, background, background trans, directbeam, directbeam trans (ess.loki)
* Transform coordinates: sample, bench, monitor position offsets (ess.loki.larmor, consider ess.loki.sans2d, dedicated loader,  helper functions)
* Masking (for file or programatically, ess.loki, masks applied as a part of loader da = ess.loki.load(file) or manually, da.masks['beam_stop'] = ess.loki.sans2d.make_beamstop_mask(da) or ess.loki.mask_beamstop(da)). Paths (not files) to mask to ess.loki.sans2d 
* Instrument geometry offset ess.loki.sans2d.load(.., offset = {position:monitor})
* Define wavelenght and q bins (wavelenght bands) geomspace is fine for the moment
* Reduction (regular full wavelenght range) - ess.sans
* Reduction by wavelenght (reduction by wavelenght through the same fucntion, ess.sans provided loki/sans2d specific functions are removed). groupby argument is LoKI specific. To be looked at
* Normalize and subtract (ess.sans) make more sense to be part of top-level function, both should be fine


### The notebook uses sans.py module that contains

* project_xy don't include 
* solid_angle (ess.loki.larmor and ess.loki.sans2d) -> candidate for scippneutron
* subtract_background_mean
* transmission_fraction - generic function taking monitors as arguments in ess.sans to be called from ess.loki.sans2d. Preprocessing of monitors can be excluded from the main function (independent helper)
* to_wavelength - rebinning may not be required, norm term may be provided as histogram. Lines 87-92 can be moved outside and the call monitor
* reduce uses groupby layer at the moment but may be different argument. Has some specific features of instrument. Spectrum may not be the only choice. ess.loki.sans2d. Eventually can be moved to ess.sans
* reduce_by_wavelength
* q resolution small clean-up (global sample variable)
* gravity - look at the issue

### sans.py calls another module contrib.py

* midpoints 
* to_bin_centers candidate for scipp, copy for now
* to_bin_edges candidate for scipp, copy for now
* map_to_bins - may be better way to implement it, copy for now
* select_bins (not used?)
* make_slices may be nicer way to implement it now


In [1]:
import scipp as sc
import scippneutron as scn
from scipp.plotting import plot
from load_files import load_isis, load_rkh_q, load_rkh_wav, setup_masks
from transform_coordinates import setup_offsets, setup_geometry
import sans
import numpy as np

## Loading files

In [2]:
try:
    import dataconfig # run make_config.py to create this
except:
    print("ERROR: dataconfig.py not find please run `make_config.py`\n")
    !python make_config.py -h # change to `-f path` or run in terminal

path = dataconfig.data_root
direct_beam_file = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'
#TODO: Are we using moderator file?
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
mantid_q1d_file = '63114rear_1D_1.75_16.5.txt'

sample_run_number = 63114
sample_transmission_run_number = 63114
background_run_number = 63159
background_transmission_run_number = 63159 
directbeam_run_number = 63091
directbeam_transmission_run_number = 63091

mask_1 = f'{path}/MASK_SANS2D_REAR_Edges_16Mar2015.xml'
mask_2 = f'{path}/MASK_SANS2D_FRONT_Edges_16Mar2015.xml'
mask_3 = f'{path}/MASK_SANS2D_BOTH_Extras_24Mar2015.xml'
mask_4 = f'{path}/MASK_SANS2D_REAR_Bottom_3_tubes_16May2014.xml'
mask_5 = f'{path}/MASK_SANS2D_beam_stop_4m_x_100mm_2July2015_medium_beamstop.xml'
mask_6 = f'{path}/MASK_SANS2D_REAR_module2_tube12.xml'
mask_7 = f'{path}/MASK_SANS2D_FRONT_module3_tube14_module5_tube6_tube7.xml'
mask_8 = f'{path}/MASK_SANS2D_FRONT_module2_tube18.xml'
mask_9 = f'{path}/MASK_SANS2D_FRONT_module5_tube20.xml'

#idf_file is not used
idf_filename = f'{path}/SANS2D_Definition_Tubes.xml'

l_collimation = sc.Variable(value=4.0, unit=sc.units.m)
r2 = sc.Variable(value=4.0/1000, unit=sc.units.m) # sample aperture radius
r1 = sc.Variable(value=10.0/1000, unit=sc.units.m) # source aperture radius (in user file its diameter)  
dr = sc.Variable(value=8.0/1000, unit=sc.units.m) # virtual ring width on detector

#Calibration file should be read in

In [3]:
%%time

direct_beam = load_rkh_wav(filename=f'{path}/{direct_beam_file}')

sample = load_isis(filename=f'{path}/SANS2D000{sample_run_number}.nxs', spectrum_size = 245760//2)
sample_trans = load_isis(filename=f'{path}/SANS2D000{sample_transmission_run_number}.nxs', spectrum_size = 245760//2)
background = load_isis(filename=f'{path}/SANS2D000{background_run_number}.nxs', spectrum_size = 245760//2)
background_trans = load_isis(filename=f'{path}/SANS2D000{background_transmission_run_number}.nxs', spectrum_size =  245760//2)
directbeam = load_isis(filename=f'{path}/SANS2D000{directbeam_run_number}.nxs', spectrum_size = 245760//2)

#TODO: What about units?

Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised units: 'frame_number'
Workspace run log 'good_frame_log' has unrecognised units: 'is_good'
Workspace run log 'period_log' has unrecognised units: 'period_number'
Workspace run log 'raw_events_log' has unrecognised units: 'events'
Workspace run log 'good_frames' has unrecognised units: 'frames'
Workspace run log 'period_change_log' has unrecognised units: 'period_number'
Workspace run log 'raw_frames' has unrecognised units: 'frames'
Workspace run log 'veto_log' has unrecognised units: 'is_vetoing'
Workspace run log 'events_log' has unrecognised units: 'events'
Workspace run log 'frame_log' has unrecognised un

CPU times: user 1min 26s, sys: 5.34 s, total: 1min 31s
Wall time: 1min 33s


In [4]:
mantid_q1d = load_rkh_q(filename=f'{path}/{mantid_q1d_file}')

## Setting up masks and geometries

In [5]:
#TODO: resolve this. Wasn't able to transform coordinates without doing this 
sample = sample['spectrum', 0:245760//2].copy()
background = background['spectrum', 0:245760//2].copy()
directbeam = sample['spectrum', 0:245760//2].copy()

In [6]:
setup_offsets(sample,background, directbeam)

In [7]:
mask_files = [mask_1, mask_2, mask_3, mask_4, mask_5, mask_6, mask_7, mask_8, mask_9]

setup_masks(idf_filename, mask_files, sample, background)                                                           
    
#PRINT masking off beamstop arm 15mm wide at 20 degrees
#!PRINT not masking beam stop arm, M4 out
#mask/line 15 20
    

In [8]:
sample = sample['spectrum', 0:122880//2].copy()
background = background['spectrum', 0:122880//2].copy()

In [9]:
sample.coords['base_position'] = sample.coords['position'].copy()
background.coords['base_position'] = background.coords['position'].copy()
directbeam.coords['base_position'] = directbeam.coords['position'].copy()

In [10]:
setup_geometry(sample, background)

## Defining parameters for data reduction 

In [11]:
q_bins = sc.geomspace(dim = 'Q', start = 0.008, stop = 0.6, num = 55, unit = sc.units.one/sc.units.angstrom)
wavelength_bins = sc.geomspace(dim = 'wavelength', start = 1.75, stop = 16.5, num = 118, unit=sc.units.angstrom)
wavelength_bands = sc.geomspace(dim = 'wavelength', start = 1.75, stop = 16.5, num = 6, unit=sc.units.angstrom)

## Reduction (full spectra)

In [12]:
%%time
sample_q_reduce = sans.to_q(data=sample,
                        transmission=sample_trans,
                        direct_beam=direct_beam,
                        direct_beam_transmission=directbeam, # note: background_trans
                        masks=sample.masks,
                        q_bins = q_bins,
                        wavelength_bins = wavelength_bins)

CPU times: user 476 ms, sys: 263 ms, total: 739 ms
Wall time: 281 ms


In [13]:
%%time
background_q_reduce = sans.to_q(data=background,
                            transmission=background_trans,
                            direct_beam=direct_beam,
                            direct_beam_transmission=directbeam, # note: background_trans
                            masks=background.masks,
                            q_bins = q_bins,
                            wavelength_bins = wavelength_bins)

CPU times: user 477 ms, sys: 283 ms, total: 760 ms
Wall time: 353 ms


In [14]:
reduced = sans.normalize_and_subtract(sample_q_reduce, background_q_reduce)

## Reduction by wavelength

In [15]:
%%time
sample_q_lambda = sans.to_q(data=sample,
                                         transmission=sample_trans,
                                         direct_beam=direct_beam,
                                         direct_beam_transmission=directbeam, # note: background_trans
                                         masks=sample.masks,
                                         q_bins = q_bins,
                                         wavelength_bins = wavelength_bins,
                                         wavelength_bands = wavelength_bands)

CPU times: user 1.25 s, sys: 673 ms, total: 1.93 s
Wall time: 959 ms


In [16]:
%%time
background_q_lambda = sans.to_q(data=background,
                                        transmission=background_trans,
                                        direct_beam=direct_beam,
                                        direct_beam_transmission=directbeam, # note: same as transmission
                                        masks=background.masks,
                                        q_bins = q_bins,
                                        wavelength_bins = wavelength_bins,
                                        wavelength_bands = wavelength_bands)

CPU times: user 1.14 s, sys: 636 ms, total: 1.77 s
Wall time: 911 ms


In [17]:
sample_q_reduce_wav = sc.sum(sample_q_lambda, 'wavelength')
background_q_reduce_wav = sc.sum(background_q_lambda, 'wavelength')

In [18]:
reduced_by_wavelength = sans.normalize_and_subtract(sample_q_reduce_wav, background_q_reduce_wav)

## Comoparison reduced vs reduced_by_wavelength

In [19]:
plot({'reduced': reduced, 'reduced_by_wavelength':reduced_by_wavelength})

In [20]:
reduced_lambda = sans.normalize_and_subtract(sample_q_lambda, background_q_lambda)
sc.plot(sc.collapse(reduced_lambda, keep='Q'))

## Comparison with Mantid

In [21]:
#scale factor at the end 0.02364 (from mantid)
reduced_scaled = sc.scalar(0.03364) * reduced
plot({'scipp': reduced_scaled, 'mantid':mantid_q1d})